Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils

import pickle
import time
import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
# category = "balanced_data"
category = "imbalanced_data"
files = glob2.glob(f'{path}R/simulated_data/{category}/*.h5')
files = [f[len(f"{path}R/simulated_data/{category}/"):-3] for f in files]
files

['data_1c8',
 'data_-1c4',
 'data_-1c8',
 'data_0c4',
 'data_0c8',
 'data_0c16',
 'data_1.5c4',
 'data_1c4',
 'data_1.5c8',
 'data_1.5c16',
 'data_-1c16',
 'data_1c16']

In [3]:
for category in ["imbalanced_data", "balanced_data"]:
    files = glob2.glob(f'{path}R/simulated_data/{category}/*.h5')
    files = [f[len(f"{path}R/simulated_data/{category}/"):-3] for f in files]


    df = pd.DataFrame()
    dropout = 0.9#0.8
    lr = 0.4
    layers = [200, 40, 60]
#     layers = [200, 100, 50, 30]
    temperature = 0.07
    for dataset in files:
        print(f">>>>> Data {dataset}")
        t0 = time.time()
        data_mat = h5py.File(f"{path}R/simulated_data/{category}/{dataset}.h5", "r")
        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])

        cluster_number = np.unique(Y).shape[0]
        nb_genes = 500
        X = train.preprocess(X, nb_genes=nb_genes)
        preprocess_time = time.time()-t0
        for run in range(3):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            start = time.time()
            dresults = train.run(X,
                                 cluster_number,
                                 dataset,
                                 Y=Y,
                                 nb_epochs=300,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 layers=layers,
                                 save_to=f"{path}output/{category}/{run}/",
                                 save_pred = True)
            time_k = (dresults["t_k"] - start) + preprocess_time
            time_l = (dresults["t_clust"] - start) + (dresults["t_l"] -dresults["t_k"]) +preprocess_time
            dresults["t_k"] = time_k
            dresults["t_l"] = time_l
            dresults["temperature"] = temperature
            dresults["dropout"] = dropout
            dresults["nb_genes"] = nb_genes
            dresults["layers"] = str(layers)
            dresults["run"] = run


            print(f"# {temperature}, {dropout}, {lr}, {layers}", 
                  dresults.get('COMBINED_kmeans_ari', ""),
                  dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                  dresults.get('leiden_ari_0', ""))
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(f"{path}output/pickle_results/{category}/{category}_baseline.pkl")

>>>>> Data data_1c8
(3000, 2500) (3000, 2500) keeping 500 genes
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
>>>>> Data data_-1c4
(3000, 2500) (3000, 2500) keeping 500 genes
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
>>>>> Data data_-1c8
(3000, 2500) (3000, 2500) keeping 500 genes
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
>>>>> Data data_0c4
(3000, 2500) (3000, 2500) keeping 500 genes
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
>>>>> Data data_0c8
(3000, 2500) (3000, 2500) keeping 500 genes
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
>>>>> Data data_0c16
(3000, 2500) (3000, 2500) keeping 500 genes
# 0.07, 0.9, 0.4, [200, 40, 60]    
# 0.07, 0.9, 0.4, [200, 40, 60]    
#